# 데이터 전처리 전략 및 파이프라인 구축

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, Normalizer
import scaling_utils

np.random.seed(1234)
X = np.random.rand(100,3) # 샘플 100개, 특성 3개
# 1번 특성과 차이를 두기
X[:,1] *= 12 # 2번 특성은 10배 이상 증가
X[:,2] += 100 # 3번 특성은 100을 더해 평균 이동 

df = pd.DataFrame(X, columns = ['Feature_A', 'Feature_B', 'Feature_C'])

X_train, X_test = train_test_split(df, test_size = 0.2, random_state=1234)

### 샘플 데이터 확인

In [2]:
from sklearn.preprocessing import FunctionTransformer
scaling_utils.analyze_scaling(FunctionTransformer(), df, 'Raw Data')


 REPORT: Raw Data
        Feature_A   Feature_B   Feature_C
count  100.000000  100.000000  100.000000
mean     0.540332    6.708500  100.468768
std      0.294869    3.289609    0.283419
min      0.002189    0.355764  100.006764
25%      0.283054    3.939651  100.225119
50%      0.603174    7.087043  100.476830
75%      0.801795    9.232592  100.702162
max      0.992081   11.935846  100.993567
--------------------------------------------------



,Feature_A,Feature_B,Feature_C
0,0.191519,7.465305,100.437728
1,0.785359,9.359710,100.272593
2,0.276464,9.622466,100.958139
3,0.875933,4.293807,100.500995
4,0.683463,8.552424,100.370251
...,...,...,...
95,0.514004,10.426451,100.720347
96,0.903591,3.331380,100.256665
97,0.699703,10.818827,100.983692
98,0.640913,3.960089,100.606675


###  데이터 상태별 전처리 파이프라인 조합 전략

| 데이터 증상 | 추천 파이프라인 조합 | 주요 목표 | 적용 모델 |
| :--- | :---: | :---: | :---: |
| **01 분포 쏠림 + 단위 불일치** | `PowerTransformer` → `StandardScaler` | 왜도 제거 후 표준화 | 선형 회귀, 로지스틱 회귀 |
| **02 이상치 과다 + 범위 제한** | `RobustScaler` → `MinMaxScaler` | 이상치 방어 후 0~1 고정 | 딥러닝(ANN), KNN |
| **03 희소 데이터 (0이 많음)** | `MaxAbsScaler` → `Normalizer` | 0의 위치 유지하며 단위 조정 | 텍스트 분석, 추천 시스템 |
| **04 기하학적 거리 중심** | `StandardScaler` → `Normalizer(L2)` | 평균 교정 후 벡터 길이 통일 | SVM, 클러스터링 |
| **05 정규분포 + 단위 불일치** | `StandardScaler` (단일 공정) | 평균 0, 분산 1로 통일 | 대부분의 ML 모델 |

In [3]:
# 01 조합 파이프라인
pipe01 = Pipeline([
    ('power', PowerTransformer(method= 'yeo-johnson')),
    ('l2', Normalizer(norm= 'l2'))
])

In [4]:
df_final = scaling_utils.analyze_scaling(pipe01, X_train, 'Pipeline01')


 REPORT: Pipeline01
       Feature_A  Feature_B     Feature_C
count  80.000000  80.000000  8.000000e+01
mean    0.013135  -0.006683 -1.794709e-16
std     0.747444   0.673621  1.934680e-15
min    -0.999874  -1.000000 -9.319917e-15
25%    -0.852289  -0.663682 -6.795669e-16
50%     0.068525   0.029476 -4.044713e-17
75%     0.725270   0.666035  7.935566e-16
max     0.999723   0.999927  4.358266e-15
--------------------------------------------------



#  최종 보고서

In [5]:
import scaling_utils
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 파이프라인 구축
pipe = Pipeline([('scaler', StandardScaler())])

# 헬퍼 함수를 이용한 최종 보고서
scaling_utils.analyze_scaling(pipe, X_train, "Final Report")


 REPORT: Final Report
          Feature_A     Feature_B     Feature_C
count  8.000000e+01  8.000000e+01  8.000000e+01
mean   6.106227e-17  7.771561e-17 -1.126876e-14
std    1.006309e+00  1.006309e+00  1.006309e+00
min   -1.728359e+00 -1.957918e+00 -1.649044e+00
25%   -1.006737e+00 -7.827085e-01 -8.783559e-01
50%    1.086671e-01  2.989927e-02 -3.730169e-02
75%    9.657957e-01  7.371126e-01  7.972790e-01
max    1.518619e+00  1.628230e+00  1.833898e+00
--------------------------------------------------



,Feature_A,Feature_B,Feature_C
0,0.851599,-0.913099,0.332198
1,-1.380265,0.809113,1.088854
2,0.990273,0.275009,-0.125894
3,1.073820,0.627705,0.770728
4,0.331380,-0.759413,-0.921765
...,...,...,...
75,-1.003024,-0.527944,-1.415579
76,0.128781,-1.344568,0.791733
77,1.147923,-1.153879,-0.444614
78,1.069153,-0.952462,1.005224


## 실험 결론

본 실험은 직접 생성한 샘플 데이터(균등 분포 기반)를 사용하여 각 스케일링 기법의 반응을 테스트하였으며, 다음과 같은 결론을 도출함.

### ✅ 성공한 조합
1. **StandardScaler (단독)**
   - **평가**: 본 샘플 데이터처럼 분포가 고른 경우, 평균(Mean)와 단위(Scale)를 맞추는 것만으로도 완벽한 전처리가 가능함.
   - **결과**: Feature A, B, C의 체급이 동일하게 정렬되어 거리 기반 모델에 즉시 투입 가능한 상태가 됨.

2. **PowerTransformer + StandardScaler (파이프라인)**
   - **평가**: 샘플 데이터에서는 과한 처방일 수 있으나, 데이터에 왜도가 발생할 경우를 대비한 가장 강력한 방어선임.
   - **결과**: 어떤 지저분한 데이터가 들어와도 정규분포화와 표준화를 동시에 수행할 수 있음을 확인함.

### ❌ 실패(부적합)한 조합 및 주의사항
1. **Normalizer (선행 배치)**
   - **사유**: `Feature_B`가 12배 크고 `Feature_C`가 100만큼 편향된 상태에서 정규화를 먼저 하면, 큰 값들이 방향성을 지배해 버려 작은 피처의 정보가 손실됨.
   - **교훈**: **반드시** 표준화(Standard)를 통해 체급을 맞춘 후 마지막에 투입해야 함.

2. **MinMaxScaler (이상치 취약성)**
   - **사유**: 본 샘플에는 이상치가 없었으나, 실전에서 이상치가 1개라도 섞일 경우 0~1 사이의 데이터 응집도가 급격히 떨어질 위험이 있음.

---
**최종 결론:** 샘플 데이터에서는 **StandardScaler**가 가장 효율적이었으나, 향후 발생할 수 있는 비정규 데이터까지 대응하기 위해 **[PowerTransformer → StandardScaler]**를 표준 공정으로 채택함.